<a href="https://colab.research.google.com/github/shenjiejie2017/test/blob/main/Michelle_Shen_Sample_Calc_Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project Introduction
This workbook will use the columns in "Input" tab of "sample_calc.xlsx" file and create 2 additional columns. The output will be a csv file with columns in the "Input" file with additional 2 columns ("fund_fv" and "fund_diff") saved in the same director of this notebook. 

After running the second cell, the note book will ask you to choose the "sample_calc.xlsx" file from your local drive to continue. So, you need to have file saved somewhere in your computer local drive before running this notebook.

I created this notebook using Google Colab. If you have a Google account, you can go to https://colab.research.google.com/ and click "Upload" to load this notebook. This notebook will be loaded to your Google Colab and ready to be used.




In [1]:
# Import libraries that are used for data manipulations here
import pandas as pd
import numpy as np
import inspect

In [2]:
from google.colab import files
uploaded = files.upload()

import io
df_Input = pd.read_excel(io.BytesIO(uploaded['sample_calc.xlsx']), "Input")
# Dataset is now stored in a Pandas Dataframe

Saving sample_calc.xlsx to sample_calc.xlsx


In [3]:
#Take a look at first 5 rows of "Input" tab
df_Input.head()

,Date,Fund Value,COI Charges,Fund Prem Coll,Load Mat/R/Pmt,Load Exc/1/%P,Load Mat/1/%P,Load Exc/R/%P,Load Mat/R/%P,Col 2,Col 3,Col 4,Col 5,Col 6,Col 7,Credited Interest
0,2020-06-30,2010.2,0.00,0,0,0.00,0.00,0,0,0.00,0,0,0,0,0,0.00
1,2020-07-31,2013.3,6.23,93,0,8.23,3.45,0,0,10.22,0,0,23,40,0,1.23
2,2020-08-31,2016.4,6.23,93,0,8.23,3.45,0,0,10.22,0,0,23,40,0,1.23
3,2020-09-30,2019.5,6.23,93,0,8.23,3.45,0,0,10.22,0,0,23,40,0,1.23
4,2020-10-31,2022.6,6.23,93,0,8.23,3.45,0,0,10.22,0,0,23,40,0,1.23


In [5]:
#take a look at how many rows and columns "input" tab has
print(f"There are {df_Input.shape[0]} rows and {df_Input.shape[1]} columns")

There are 13 rows and 16 columns


In [7]:
#create a OOP class to do data manipulation
class Calculation(object):
    
    def __init__(self, df):
        #attributes of existing columns
        self.date = df['Date']
        self.fund_value=df["Fund Value"]
        self.coi_charges=df['COI Charges']
        self.fund_prem_coll=df['Fund Prem Coll']
        self.load_mat_r_pmt=df['Load Mat/R/Pmt']
        self.load_exc_1_p=df['Load Exc/1/%P']
        self.load_mat_1_p=df['Load Mat/1/%P']
        self.load_exc_r_p=df['Load Exc/R/%P']
        self.load_mat_r_p=df['Load Mat/R/%P']
        self.col2=df['Col 2']
        self.col3=df['Col 3']
        self.col4=df['Col 4']
        self.col5=df['Col 5']
        self.col6=df['Col 6']
        self.col7=df['Col 7']
        self.credited_int=df['Credited Interest']

        #attributes to be created, initialize them first
        self.fund_fv=np.zeros(df.shape[0])  #initialize fund_fv to be zeros
        self.fund_diff=np.ones(df.shape[0])  #can't intilize fund_diff to be zeros since this is the final results I wnat. I initialize fund_diff to be ones
      
    #class function/methods for fund_fv column
    def get_fund_fv(self):
        #first row need special coding
        self.fund_fv[0] = self.fund_value[0]
 
        #interate through the rest rows
        for i in range(1,13):
            self.fund_fv[i] = self.fund_fv[i-1] + self.fund_prem_coll[i] + self.credited_int[i] -self.load_exc_1_p[i] -self.load_mat_1_p[i] -self.load_exc_r_p[i] - self.load_mat_r_p[i]- self.col2[i] - self.col3[i]- self.col4[i]- self.col5[i]- self.col6[i]- self.col7[i]-self.load_mat_r_pmt[i] -self.coi_charges[i]
    
    #class function/methods for fund_diff column
    def get_diff(self):
        
           self.fund_diff = np.round(self.fund_value-self.fund_fv,2)
      

In [8]:
#creat an instant "my_object" of Calculation class
my_object = Calculation(df_Input)

In [9]:
#create "fund_fv" column
my_object.get_fund_fv()

In [10]:
#create "get_diff()" column
my_object.get_diff()

In [13]:
#create a function to put columns needed for final output to dictionary 

def props(obj):
    pr = {}
    for name in dir(obj):
        value = getattr(obj, name)
        if not name.startswith('__') and not inspect.ismethod(value):
            pr[name] = value
    return pr

In [14]:
#creat a dictionary to put all columns needed for output
my_dictionary= props(my_object)

In [15]:
#put dictionary to DataFrame
df_result = pd.DataFrame(my_dictionary)

In [17]:
#change the DataFrame column order to the order I want
df_result=df_result[[ 'date', 'fund_value', 'coi_charges', 'fund_prem_coll', 'load_mat_r_pmt', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'credited_int',   'fund_fv','fund_diff'    ]]

In [19]:
#Take a look at result data frame
#Here since we have only 13 records, I want to see all of them so I use head(20). Pandas will ignore the extrac rows requsted if the rows in head() exceed the total record numbers
df_result.head(20)

,date,fund_value,coi_charges,fund_prem_coll,load_mat_r_pmt,col2,col3,col4,col5,col6,col7,credited_int,fund_fv,fund_diff
0,2020-06-30,2010.2,0.00,0,0,0.00,0,0,0,0,0,0.00,2010.2,0.0
1,2020-07-31,2013.3,6.23,93,0,10.22,0,0,23,40,0,1.23,2013.3,-0.0
2,2020-08-31,2016.4,6.23,93,0,10.22,0,0,23,40,0,1.23,2016.4,-0.0
3,2020-09-30,2019.5,6.23,93,0,10.22,0,0,23,40,0,1.23,2019.5,-0.0
4,2020-10-31,2022.6,6.23,93,0,10.22,0,0,23,40,0,1.23,2022.6,-0.0
5,2020-11-30,2025.7,6.23,93,0,10.22,0,0,23,40,0,1.23,2025.7,-0.0
6,2020-12-31,2028.8,6.23,93,0,10.22,0,0,23,40,0,1.23,2028.8,-0.0
7,2021-01-31,2031.9,6.23,93,0,10.22,0,0,23,40,0,1.23,2031.9,-0.0
8,2021-02-28,2035.0,6.23,93,0,10.22,0,0,23,40,0,1.23,2035.0,-0.0
9,2021-03-31,2038.1,6.23,93,0,10.22,0,0,23,40,0,1.23,2038.1,-0.0


In [20]:
df_result.to_csv(r"Michelle_Shen_Deloitte_answer.csv")  